### A1 Part-2 Classification
#### Probing

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import accelerate

from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

##### Dataset Preparation

In [112]:
dataset = pd.read_csv('../data/data.csv')
dataset.drop_duplicates(subset=['Sentence'], inplace=True)
dataset.reset_index(drop=True, inplace=True)
dataset = dataset.iloc[:250]
dataset

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
...,...,...
245,RT @stockdemons think shorting here into close...,negative
246,VDW combined with LXE devices enhances product...,positive
247,"Thus the method will cut working costs , and w...",positive
248,Talentum 's net sales in September were smalle...,negative


In [93]:
dataset.columns

Index(['Sentence', 'Sentiment'], dtype='object')

In [113]:
print(f'No. of samples: {len(dataset)}')
print(f'No. of features: {dataset.drop(columns="Sentiment").shape[1]}')
print(f'Features: {dataset.drop(columns="Sentiment").columns.to_list()}')
print(f'No. of classes: {len(dataset["Sentiment"].unique())}')
print(f'Classes: {dataset["Sentiment"].unique()}')

No. of samples: 250
No. of features: 1
Features: ['Sentence']
No. of classes: 3
Classes: ['positive' 'negative' 'neutral']


In [114]:
X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=['Sentiment']), dataset['Sentiment'], test_size=.2)

X_train = X_train.astype(str)
X_test = X_test.astype(str)
y_train = y_train.astype(str)
y_test = y_test.astype(str)

##### LLM Preparation

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type='nf8',  # Can be 'nf4' or 'fp4'
    bnb_8bit_compute_dtype=torch.bfloat16  # Adjust compute type if needed
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto',
    quantization_config=quantization_config,
    output_hidden_states=True,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [7]:
def get_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    total_size_bytes = total_params * 2
    total_size_gb = total_size_bytes / (1024 ** 3)
    print(f"Model size: {total_size_gb:.2f} GB")

get_model_size(model)

Model size: 14.96 GB


In [8]:
model.device

device(type='cuda', index=0)

In [98]:
def prompt_llm(prompt, max_tokens=20):
    messages = [
        {"role": "system", "content": "You are a sentiment predicting chatbot that predicts the sentiment of a financial sentence as positive, negative or neutral."},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=max_tokens,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            output_hidden_states=True,  # Enable hidden states
            return_dict_in_generate=True  # Ensure hidden states are included in the output
        )

    generated_tokens = outputs.sequences[0][input_ids.shape[-1]:]
    
    hidden_states = outputs.hidden_states  # Shape: (num_layers, batch_size, seq_len, hidden_size)
    num_layers = len(hidden_states)
    # return hidden_states
    
    # Last Token Embedding
    first_layer_hidden_state = hidden_states[-1][0][:, -1, :].squeeze().cpu().numpy()
    middle_layer_hidden_state = hidden_states[-1][num_layers // 2][:, -1, :].squeeze().cpu().numpy()
    last_layer_hidden_state = hidden_states[-1][-1][:, -1, :].squeeze().cpu().numpy()

    response_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    del input_ids
    torch.cuda.empty_cache()

    return {
        'prompt': prompt,
        'response': response_text,
        'first_layer_embedding': first_layer_hidden_state,  # Embedding from the first layer
        'middle_layer_embedding': middle_layer_hidden_state,  # Embedding from the middle layer
        'last_layer_embedding': last_layer_hidden_state  # Embedding from the last layer
    }

##### Extracting Embeddings

In [115]:
train_embeddings_first = []
train_embeddings_middle = []
train_embeddings_last = []
train_prompts = []
train_responses = []

for x in tqdm(X_train.iterrows(), total=len(X_train), desc='Generating embeddings for training data'):
    prompt = f'What is the sentiment for this financial sentence: Just output Postive, Negative or Neutral?\n"{x[1]["Sentence"]}"'
    output = prompt_llm(prompt)
    train_prompts.append(output['prompt'])
    train_responses.append(output['response'])
    train_embeddings_first.append(output['first_layer_embedding'])
    train_embeddings_middle.append(output['middle_layer_embedding'])
    train_embeddings_last.append(output['last_layer_embedding'])

train_embeddings_first = np.array(train_embeddings_first)
train_embeddings_middle = np.array(train_embeddings_middle)
train_embeddings_last = np.array(train_embeddings_last)

Generating embeddings for training data: 100%|██████████| 200/200 [01:32<00:00,  2.15it/s]


In [116]:
test_embeddings_first = []
test_embeddings_middle = []
test_embeddings_last = []
test_prompts = []
test_responses = []

for x in tqdm(X_test.iterrows(), total=len(X_test), desc='Generating embeddings for testing data'):
    prompt = f'What is the sentiment for this financial sentence: Just output Postive, Negative or Neutral?\n"{x[1]["Sentence"]}"'
    output = prompt_llm(prompt)
    test_prompts.append(output['prompt'])
    test_responses.append(output['response'])
    test_embeddings_first.append(output['first_layer_embedding'])
    test_embeddings_middle.append(output['middle_layer_embedding'])
    test_embeddings_last.append(output['last_layer_embedding'])

test_embeddings_first = np.array(test_embeddings_first)
test_embeddings_middle = np.array(test_embeddings_middle)
test_embeddings_last = np.array(test_embeddings_last)

Generating embeddings for testing data: 100%|██████████| 50/50 [00:23<00:00,  2.17it/s]


##### Probing Classifier

In [117]:
cl_model_first = RandomForestClassifier()
cl_model_middle = RandomForestClassifier()
cl_model_last = RandomForestClassifier()

In [118]:
y_train = y_train.to_numpy(dtype='object')
y_test = y_test.to_numpy(dtype='object')

In [119]:
cl_model_first.fit(train_embeddings_first, y_train)
cl_model_middle.fit(train_embeddings_middle, y_train)
cl_model_last.fit(train_embeddings_last, y_train)

RandomForestClassifier()

##### Evaluation

Training Accuracy

In [120]:
y_pred_train_first = cl_model_first.predict(train_embeddings_first)
y_pred_train_middle = cl_model_middle.predict(train_embeddings_middle)
y_pred_train_last = cl_model_last.predict(train_embeddings_last)

y_train, y_pred_train_first, y_pred_train_middle, y_pred_train_last

(array(['negative', 'positive', 'neutral', 'positive', 'neutral',
        'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
        'positive', 'positive', 'negative', 'neutral', 'negative',
        'negative', 'neutral', 'neutral', 'positive', 'neutral', 'neutral',
        'neutral', 'neutral', 'positive', 'positive', 'neutral', 'neutral',
        'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive',
        'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'negative',
        'negative', 'positive', 'neutral', 'neutral', 'positive',
        'negative', 'neutral', 'positive', 'negative', 'neutral',
        'neutral', 'positive', 'neutral', 'negative', 'positive',
        'neutral', 'neutral', 'positive', 'negative', 'neutral',
        'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
        'positive', 'positive', 'neutral', 'neutral', 'neutral',
        'positive', 'positive', 'neutral', 'positive', 'neutral',
        'neutral', '

In [121]:
print(f'First Layer Accuracy: {accuracy_score(y_train, y_pred_train_first)}')
print(f'Middle Layer Accuracy: {accuracy_score(y_train, y_pred_train_middle)}')
print(f'Last Layer Accuracy: {accuracy_score(y_train, y_pred_train_last)}')

print(f'First Layer F1 Score: {f1_score(y_train, y_pred_train_first, average="weighted")}')
print(f'Middle Layer F1 Score: {f1_score(y_train, y_pred_train_middle, average="weighted")}')
print(f'Last Layer F1 Score: {f1_score(y_train, y_pred_train_last, average="weighted")}')

First Layer Accuracy: 0.61
Middle Layer Accuracy: 1.0
Last Layer Accuracy: 1.0
First Layer F1 Score: 0.5757829484413991
Middle Layer F1 Score: 1.0
Last Layer F1 Score: 1.0


Testing Accuracy

In [122]:
y_pred_test_first = cl_model_first.predict(test_embeddings_first)
y_pred_test_middle = cl_model_middle.predict(test_embeddings_middle)
y_pred_test_last = cl_model_last.predict(test_embeddings_last)

y_test, y_pred_test_first, y_pred_test_middle, y_pred_test_last

(array(['negative', 'neutral', 'positive', 'neutral', 'neutral',
        'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
        'positive', 'neutral', 'neutral', 'positive', 'negative',
        'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral',
        'negative', 'neutral', 'positive', 'negative', 'negative',
        'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive',
        'positive', 'positive', 'neutral', 'neutral', 'positive',
        'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive',
        'negative', 'neutral', 'positive', 'positive', 'neutral',
        'neutral'], dtype=object),
 array(['negative', 'negative', 'positive', 'neutral', 'positive',
        'positive', 'neutral', 'positive', 'neutral', 'positive',
        'neutral', 'positive', 'negative', 'positive', 'positive',
        'negative', 'positive', 'negative', 'negative', 'neutral',
        'positive', 'positive', 'negative', 'positive', 'positive

In [123]:
print(f'First Layer Accuracy: {accuracy_score(y_test, y_pred_test_first)}')
print(f'Middle Layer Accuracy: {accuracy_score(y_test, y_pred_test_middle)}')
print(f'Last Layer Accuracy: {accuracy_score(y_test, y_pred_test_last)}')

print(f'First Layer F1 Score: {f1_score(y_test, y_pred_test_first, average="weighted")}')
print(f'Middle Layer F1 Score: {f1_score(y_test, y_pred_test_middle, average="weighted")}')
print(f'Last Layer F1 Score: {f1_score(y_test, y_pred_test_last, average="weighted")}')

First Layer Accuracy: 0.62
Middle Layer Accuracy: 0.72
Last Layer Accuracy: 0.82
First Layer F1 Score: 0.5810038610038609
Middle Layer F1 Score: 0.7178666666666667
Last Layer F1 Score: 0.8056107539450613
